In [3]:
from model_xray.zenml.pipelines.data_creation.data_classes import ModelRepos
from model_xray.zenml.pipelines.data_creation.model_attack import embed_payload_into_pretrained_weights_pipeline
from model_xray.config_classes import EmbedPayloadConfig, EmbedType, GrayscaleLastMBytesConfig, ImageRepConfig, ImageType, PayloadType, XLSBAttackConfig
from model_xray.utils.image_rep_utils import image_rep_map

from model_xray.zenml.pipelines.data_creation.image_representation import image_representation_from_attacked_pretrained_pipeline


pretrained_model_name = "MobileNet"
pretrained_model_repo = ModelRepos.KERAS

embedding_config = EmbedPayloadConfig(
    embed_type=EmbedType.X_LSB_ATTACK_FILL,
    embed_proc_config=XLSBAttackConfig(
        x=8,
        fill=True,
        msb=False,
        payload_type=PayloadType.RANDOM,
    )
)

res = image_representation_from_attacked_pretrained_pipeline(
    pretrained_model_name=pretrained_model_name,
    pretrained_model_repo=pretrained_model_repo,
    embed_payload_config=embedding_config,
    image_rep_config=ImageRepConfig(
        image_type=ImageType.GRAYSCALE_LAST_M_BYTES,
        image_rep_config=GrayscaleLastMBytesConfig(m=4)
    )
)

Starting embedding random payload into MobileNet pretrained weights using x_lsb_attack_fill embedding with x=8
Initiating a new run for the pipeline: image_representation_from_attacked_pretrained_pipeline.
Executing a new run.
Using user: default
Using stack: custom-stack
  experiment_tracker: mlflow_experiment_tracker
  orchestrator: default
  artifact_store: custom_artifact_store_local
Dashboard URL: http://127.0.0.1:8237/runs/0ee62796-714b-4cfa-9c79-e9d2be37c33a
Caching enabled explicitly for fetch_pretrained.
Using cached version of fetch_pretrained.
Step fetch_pretrained has started.
Caching enabled explicitly for extract_weights.
Using cached version of extract_weights.
Step extract_weights has started.
Using cached version of embed_payload_into_weights.
Step embed_payload_into_weights has started.
Using cached version of create_image_representation.
Step create_image_representation has started.
Pipeline run has finished in 1.251s.


In [6]:
# res.steps.keys()
im = res.steps["create_image_representation"].output.load()
im.shape
# im = 

(1, 4126, 4126)